In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import ipywidgets
import matplotlib.gridspec as gridspec
import matplotlib.animation as manimation
from IPython.display import display

# Explanation

An interactive widget to plot the density profiles.

In [2]:
### Load in the descriptors of the bar data
allbardata = pd.read_csv("data/bardata_allexperiments_extended.csv")

#### experiment and bar labels as strings
allbardata['Experiment'] = allbardata['Experiment'].astype(str)
allbardata['Bar'] = allbardata['Bar'].astype(str)
allbardata.set_index(['Experiment', 'Bar'], inplace=True)

###### get the intended printed length (these are multiples of 10 micron. The 1.084 comes from the resolution of the image (pixels per micron))
allbardata['Length_micron'] = np.round((allbardata['Length']/1.084/20))*20

##### drop the square bars and the very long ones
allbardata.drop( allbardata.index[allbardata['Length_micron']==40], inplace=True)
allbardata.drop( allbardata.index[allbardata['Length_micron']>330], inplace=True)

#### Load in all the density profiles
### and do light processing (smoothing)
avgn = 11 # averaging window

density_profiles = {}
for exp, bar in allbardata.index:
    filename = f'data/bars_density_profiles/{exp}_bar{bar}_threxp_densityhigh_n.txt'
    xrho = np.loadtxt(filename) ## first row: x values, the rest is density per timepoint
    x = xrho[0]
    rho = xrho[1:]
    rho_smooth = np.zeros_like(rho)

    for i in range(rho.shape[1]): #### iterate over spatial positions
        rh_extended = np.hstack((avgn//2 * [rho[0,i]], rho[:,i], avgn//2 * [rho[-1,i]]))
        rho_smooth[:,i] = np.convolve(rh_extended, np.ones(avgn)/avgn, mode='same')[avgn//2: - avgn//2+1]

    ### save to dictionary
    density_profiles[(exp,bar)] = (x,rho,rho_smooth)



In [4]:
def plot_profile(exp, bar, frame=0):
    fig, axes = plt.subplots(1, 2, figsize=(10, 4), width_ratios=[1, 2])

    ### left: profile at this frame
    x, rho, rho_smooth = density_profiles[(exp, bar)]

    axes[0].plot(x, rho[frame], label='original density')
    axes[0].plot(x, rho_smooth[frame], label='smoothed in time')
    axes[0].set_xlabel("Position (µm)")
    axes[1].set_ylabel("Density (cells/µm)")
    axes[0].set_title(f"Frame {frame}")
    axes[0].legend()

    ### right: kymo
    t = np.arange(rho.shape[0])
    pp=axes[1].pcolormesh(t, x, rho_smooth.T)
    fig.colorbar(pp, label='Density')
    axes[1].set_xlabel("Time (frames)")
    axes[1].set_ylabel("Position (µm)")
    ### add line for the current frame
    axes[1].axvline(frame, color='tab:red')
ipywidgets.interact(plot_profile, exp = allbardata.index.get_level_values(0).unique(), \
                    bar = allbardata.index.get_level_values(1).unique(), \
                   frame = (0, 200))

interactive(children=(Dropdown(description='exp', options=('20240311', '20240317', '20240320', '20240404', '20…

<function __main__.plot_profile(exp, bar, frame=0)>